# **PRIMERA PARTE: TRAER LOS DATOS DE LA PAGINA WEB** #


## TRAIGO LOS DATOS Y LOS CONVIERTO EN DATAFRAME ## 


In [68]:
import pandas as pd
import numpy as np
print( '¡Hola, Proyecto Final del Curso!')

¡Hola, Proyecto Final del Curso!


In [69]:
# importing necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

url = 'https://pipiwiki.com/wiki/List_of_M_postal_codes_of_Canada'
df = pd.read_html(url)
df_trt=df[0]
df_trt.head(5)



,Postal Code,Community,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## 1.-RENOMBRO  COLUMNAS Postal Code y Community y Neighbourhood (mal escrita la última).  ##
## 2.-Elimino los municipios 'Not Assigned' ##
## 3.-Si una celda tiene un municipio, pero un vecindario No asignado, entonces el vecindario será el mismo que el municipio. ##

In [70]:
# 1.- renombro
df_trt.rename(columns = {'Postal Code':'PostalCode','Community':'Borough','Neighbourhood':'Neighborhood'}, inplace = True) 
#2.- elimino distritos not assigne
df_trt.drop(df_trt[df_trt['Borough'] == 'Not assigned'].index, inplace=True)
#3.- igualo municipio vecindario si este es not assigned
df_trt.loc[df_trt['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_trt['Borough']
df_trt.head(8)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills


## CANTIDAD DE REGISTROS DEL DATAFRAME ##

In [71]:
df_trt.shape

(103, 3)

# **SEGUNDA PARTE: AGREGAR COORDENADAS** #

## BUSCO LAS COORDENADAS EN LABORATORIO ##

In [72]:
GeoSpatial_URL ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"

df_ge = pd.read_csv(GeoSpatial_URL)
df_ge.head(5)
#coordinates=pd.read_csv('Geospatial_Coordinates.csv')

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## REALIZO EL MERGE Y BORRO COLUMNA DUPLICADA ##

In [73]:
df_trt=pd.merge(left=df_trt, right=df_ge, how='left', left_on='PostalCode', right_on='Postal Code')
df_trt=df_trt.drop(columns='Postal Code')
df_trt.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## CANTIDAD DE REGISTROS IGUAL AL ANTERIOR ##

In [74]:
df_trt.shape

(103, 5)

# **TERCERA PARTE: AGREGAR COORDENADAS** #

## CARGO LIBRERIAS NECESARIAS ##

In [75]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # librería para manejar archivos JSON 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud

import requests # librería para manejar solicitudes
from pandas.io.json import json_normalize # librería para convertir un archivo json en un dataframe pandas

# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors

# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans

!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
import folium # librería para graficar mapas 

print('Libraries imported.')

Solving environment: ...working... 
  - anaconda/win-64::ca-certificates-2022.3.29-haa95532_1, anaconda/win-64::certifi-2021.10.8-py39haa95532_2, anaconda/win-64::openssl-1.1.1n-h2bbff1b_0
  - anaconda/win-64::ca-certificates-2022.3.29-haa95532_1, anaconda/win-64::openssl-1.1.1n-h2bbff1b_0, defaults/win-64::certifi-2021.10.8-py39haa95532_2
  - anaconda/win-64::certifi-2021.10.8-py39haa95532_2, anaconda/win-64::openssl-1.1.1n-h2bbff1b_0, defaults/win-64::ca-certificates-2022.3.29-haa95532_1
  - anaconda/win-64::openssl-1.1.1n-h2bbff1b_0, defaults/win-64::ca-certificates-2022.3.29-haa95532_1, defaults/win-64::certifi-2021.10.8-py39haa95532_2
  - anaconda/win-64::certifi-2021.10.8-py39haa95532_2, defaults/win-64::ca-certificates-2022.3.29-haa95532_1, defaults/win-64::openssl-1.1.1n-h2bbff1b_0
  - defaults/win-64::ca-certificates-2022.3.29-haa95532_1, defaults/win-64::certifi-2021.10.8-py39haa95532_2, defaults/win-64::openssl-1.1.1n-h2bbff1b_0
  - anaconda/win-64::ca-certificates-2022.3.29

## REVISO CANTIDAD DE VECINDARIOS Y MUNICIPIOS ##

In [76]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(df_trt['Borough'].unique()),
        df_trt.shape[0]
    )
)
print (df_trt['Borough'].unique())

Toronto has 10 boroughs and 103 neighborhoods.
['North York' 'Downtown Toronto' 'Etobicoke' 'Scarborough' 'East York'
 'York' 'East Toronto' 'West Toronto' 'Central Toronto' 'Mississauga']


## OBTENGO COORDENADAS GRAFICAS DE TORONTO Y CREO EL MAPA ##

In [77]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# crear un mapa de Toronto utilizando los valores de latitud y longitud
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# añado marcadores al mapa
for lat, lng, borough, neighborhood in zip(df_trt['Latitude'], df_trt['Longitude'],df_trt['Borough'], df_trt['Neighborhood']):
#for lat, lng,  neighborhood in zip(df_trt['Latitude'], df_trt['Longitude'],df_trt['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto



The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## OBTENGO MAPAS DE DISTINTOS DISTRITOS ##

### Mapa de North York ###

In [78]:

  
  nb='North York'
  
  df_nyo = df_trt[df_trt['Borough'] == nb].reset_index(drop=True)
  print(nb,' has  {} neighborhoods.'.format(
  len( df_trt[df_trt['Borough'] == nb].reset_index(drop=True)),
  df_trt.shape[1]    )
  )
 
  address=nb
  geolocator = Nominatim(user_agent="ny_explorer")
  location = geolocator.geocode(address)
  latitude = location.latitude
  longitude = location.longitude
  print('The geograpical coordinate of nb are {}, {}.'.format(latitude, longitude))

# crear un mapa de North York a utilizando los valores de latitud y longitud
  map_nyo = folium.Map(location=[latitude, longitude], zoom_start=11)

   
    
# añadir los marcadores al mapa
  for lat, lng, label in zip(df_nyo['Latitude'], df_nyo['Longitude'], df_nyo['Neighborhood']):
     label = folium.Popup(label, parse_html=True)
     folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyo)  
    
map_nyo

North York  has  24 neighborhoods.
The geograpical coordinate of nb are 43.7543263, -79.44911696639593.


### Mapa de Downtown Toronto ###

In [79]:

  
  nb='Downtown Toronto'
  
  df_nyo = df_trt[df_trt['Borough'] == nb].reset_index(drop=True)
  print(nb,' has  {} neighborhoods.'.format(
  len( df_trt[df_trt['Borough'] == nb].reset_index(drop=True)),
  df_trt.shape[1]    )
  )
 
  address=nb
  geolocator = Nominatim(user_agent="ny_explorer")
  location = geolocator.geocode(address)
  latitude = location.latitude
  longitude = location.longitude
  print('The geograpical coordinate of nb are {}, {}.'.format(latitude, longitude))

# crear un mapa de North York a utilizando los valores de latitud y longitud
  map_nyo = folium.Map(location=[latitude, longitude], zoom_start=11)

   
    
# añadir los marcadores al mapa
  for lat, lng, label in zip(df_nyo['Latitude'], df_nyo['Longitude'], df_nyo['Neighborhood']):
     label = folium.Popup(label, parse_html=True)
     folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyo)  
    
map_nyo

Downtown Toronto  has  19 neighborhoods.
The geograpical coordinate of nb are 43.6541457, -79.38079276646786.
